In [221]:
import numpy as np
import os
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D
from numpy.linalg import norm

In [223]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    GlobalMaxPool2D()
])

In [224]:
def extract_features_from_images(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_expand_dim = np.expand_dims(img_array, axis=0)
    img_preprocess = preprocess_input(img_expand_dim)
    result = model.predict(img_preprocess).flatten()
    norm_result = result / norm(result)
    return norm_result

In [225]:
categories = ["bottomwear", "topwear", "shoes", "accessories"]
image_features = []
filenames = []

In [226]:
base_path = "C:/Users/asus/Downloads/Styling suggestions/"
for category in categories:
    folder_path = os.path.join(base_path, category)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        filenames.append(file_path)


In [227]:
os.makedirs("features", exist_ok=True)
pkl.dump(image_features, open("features/Images_features.pkl", "wb"))
pkl.dump(filenames, open("features/filenames.pkl", "wb"))

In [229]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [230]:
labels = []
features = []

In [231]:
for category in categories:
    folder_path = f"C:\\Users\\asus\\Downloads\\Styling suggestions\\{category}"  # Double backslashes
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        labels.append(category)
        features.append(extract_features_from_images(file_path, model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [232]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

In [239]:
svm_classifier = SVC(probability=True, kernel='linear')
svm_classifier.fit(X_train, y_train)

# Save the classifier and label encoder
os.makedirs("models", exist_ok=True)
pkl.dump(svm_classifier, open("models/category_classifier.pkl", "wb"))
pkl.dump(label_encoder, open("models/label_encoder.pkl", "wb"))

In [245]:
import pickle as pkl
import numpy as np
from tensorflow.keras.preprocessing import image
from numpy.linalg import norm

# Load saved models and features
svm_classifier = pkl.load(open("models/category_classifier.pkl", "rb"))
label_encoder = pkl.load(open("models/label_encoder.pkl", "rb"))
database_features = pkl.load(open("features/Images_features.pkl", "rb"))
database_filenames = pkl.load(open("features/filenames.pkl", "rb"))

# Function to extract features from images
def extract_features_from_images(image_path, model):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_expand_dim = np.expand_dims(img_array, axis=0)
        features = model.predict(img_expand_dim)
        return features.flatten()
    except Exception as e:
        print(f"Error in feature extraction: {e}")
        raise

# Function for classification and recommendations
def classify_and_recommend(image_path, model, classifier, encoder, db_features, db_filenames):
    try:
        # Extract features from uploaded image
        features = extract_features_from_images(image_path, model)

        # Classify the image into one of the categories
        category_idx = classifier.predict([features])[0]
        category = encoder.inverse_transform([category_idx])[0]
        print(f"Classified as: {category}")

        # Find recommendations from other categories
        other_categories = [cat for cat in encoder.classes_ if cat != category]
        recommendations = {}

        for other_category in other_categories:
            # Find indices of files belonging to the current category
            indices = [i for i, fname in enumerate(db_filenames) if other_category in fname]

            # Skip processing if no files match
            if not indices:
                print(f"No matching files found for category: {other_category}")
                continue

            # Validate indices before accessing db_features
            try:
                category_features = np.array([db_features[i] for i in indices])
                category_filenames = [db_filenames[i] for i in indices]
            except IndexError as e:
                print(f"IndexError: {e}")
                print(f"Invalid indices: {indices}")
                continue

            # Compute cosine similarity
            try:
                similarities = np.dot(category_features, features) / (
                    norm(category_features, axis=1) * norm(features)
                )
                top_indices = similarities.argsort()[-3:][::-1]  # Top 3 matches
                recommendations[other_category] = [category_filenames[i] for i in top_indices]
            except Exception as e:
                print(f"Error during cosine similarity computation: {e}")
                continue

        return category, recommendations

    except Exception as e:
        print(f"Error in classify_and_recommend: {e}")
        raise

# Example usage
uploaded_image = r"C:\Users\asus\Downloads\Styling suggestions\uploads\ring_080.jpg"


# Assuming `model` is already defined as the feature extraction model
try:
    category, recommendations = classify_and_recommend(
        uploaded_image, model, svm_classifier, label_encoder, database_features, database_filenames
    )

    print("Recommendations:")
    for rec_category, rec_files in recommendations.items():
        print(f"{rec_category}: {rec_files}")
except Exception as e:
    print(f"Error during processing: {e}")
       



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Classified as: accessories
IndexError: list index out of range
Invalid indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
IndexError: list index out of range
Invalid indices: [112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184]
IndexError: list index out of range
Invalid indices: [45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 10